# Flow Routing Lesson 4.
You can run this notebook on your laptop with landlab v2.6.0 or in lab.openearthscape using the CSDMS environment.

# 1. PriorityFloodFlowRouter and a simple landscape evolution model.
- Let's now try a simple landscape evolution model (inspired from Benjamin Campforts):
    - which vertically uplifts at a constant and uniform rate.
    - which is subject to river incision, sediment transport and deposition. We'll use to this end the SpaceLargeScaleEroder, developed by Benjamin Campforts based on the SpaceScaleEroder [Shobe et al., 2017](https://doi.org/10.5194/gmd-10-4577-2017).
    - with one-to-one flow (the SpaceScaleEroder and SpaceLargeScaleEroder don't work yet on one-to-multiple flows).
- We first import the packages and components.

In [ ]:
# Import packages necessary for this lesson
import numpy as np # Array handling.
import matplotlib as mpl
import matplotlib.pyplot as plt # Graphic display
from tqdm import tqdm # to display progression line

from landlab import RasterModelGrid # Grid management.

# Depression and flow handling
from landlab.components import (
    FlowDirectorSteepest,
    FlowDirectorD8,
    PriorityFloodFlowRouter,
    SpaceLargeScaleEroder,
    BedrockLandslider  # BedrockLandslider model
)

# Flood status.
from landlab.components.depression_finder.floodstatus import FloodStatus

# import landlab plotting functionality.
from landlab import imshowhs_grid, imshow_grid
from landlab.plot.drainage_plot import drainage_plot
from landlab.plot import graph

- We then initiate the grid and the model:

In [ ]:
# Creation of the grid.
g = RasterModelGrid((15, 15), xy_spacing=10.0)

# Set of the initial topography, with random small noise
z = g.add_zeros("topographic__elevation", at="node")
g.at_node["topographic__elevation"][g.core_nodes] += np.random.rand(
    g.number_of_core_nodes
)

# Set soil depth
s = g.add_zeros("soil__depth", at="node", dtype=float)
g.at_node["soil__depth"][g.core_nodes] += 0.5
g.at_node["topographic__elevation"] += g.at_node["soil__depth"]

# Set the flow router component.
fr = PriorityFloodFlowRouter(g, flow_metric="D8", update_flow_depressions=True)
fr.run_one_step()

# Set the river incision, sediment transport and deposition component.
ha = SpaceLargeScaleEroder(g, K_sed=0.0001, K_br=0.0001, phi=0.3, H_star=1)

br = g.at_node["bedrock__elevation"] # created by the SpaceLargeScaleEroder

# Time step. 
space_dt = 250

# Display of the initial drainage network.
drainage_plot(g, surf_cmap="terrain")
plt.show()

- And now we run the model for some iterations.

In [ ]:
# Run the model
z_ori = np.array(z)
for i in tqdm(range(100)):
    # Bedrock Uplift
    br[g.core_nodes] += 0.001 * space_dt
    # Elevation update
    z[g.core_nodes] = br[g.core_nodes] + s[g.core_nodes]
    # Run the PriorityFloodFlowRouter
    fr.run_one_step()
    # Run the SpaceLargeScaleEroder
    ha.run_one_step(dt=space_dt)

# Display the finale drainage network.
drainage_plot(g, surf_cmap="terrain")
plt.show()

## 2. Model including landslides.
- Let's now try a model on a larger grid and which includes landsliding.
    - In addition to previously shown components, we will use the **BedrockLandslider** component, developed by Benjamin Campforts [Campforts et al., 2022](https://doi.org/10.1029/2022JF006745), [Campforts et al., 2020](https://doi.org/10.5194/gmd-13-3863-2020). See the Hylands tutorial in the landlab tutorial landscape evolution folder (https://github.com/landlab/landlab/blob/master/notebooks/tutorials/landscape_evolution/hylands/HyLandsTutorial.ipynb).
    - We increase the size of the grid.
    
- First, we initiate the model:

In [ ]:
# Initiate grid.
g = RasterModelGrid((40, 40), 25.)

# Create initial elevation, soil depth, bedrock elevation.
g.add_zeros("node", "topographic__elevation")
g.add_zeros("node", "soil__depth")
g.add_zeros("bedrock__elevation", at="node")

# set constant random seed for consistent topographic roughness
np.random.seed(seed=5000)
# add topographic roughness
random_noise = np.random.rand(len(g.node_y)) / 1000.0

g.at_node["bedrock__elevation"] += random_noise
g.at_node["soil__depth"][g.core_nodes] = 1.0  # soil (=sediment) only on core nodes.
g.at_node["topographic__elevation"] = g.at_node["bedrock__elevation"] + g.at_node["soil__depth"]

# Let open all boundaries
g.set_closed_boundaries_at_grid_edges(
    bottom_is_closed=False,
    left_is_closed=False,
    right_is_closed=False,
    top_is_closed=False,
)

# Initiate the components of the landscape evolution model
fr = PriorityFloodFlowRouter(g,
    flow_metric="D8",
    separate_hill_flow=True, # important to create a field hill_flow__receiver_node  required by the BedrockLandslider
    hill_flow_metric="Quinn",
    update_hill_flow_instantaneous=True,
    )
fr.run_one_step()
sp = SpaceLargeScaleEroder(
    g,
    K_sed=2.5e-5,
    K_br=2.5e-5,
    F_f=0.0,
    phi=0.0,
    H_star=1.0,
    v_s=1,
    m_sp=0.5,
    n_sp=1.0,
    sp_crit_sed=0,
    sp_crit_br=0,
)
hy = BedrockLandslider(
    g,
    angle_int_frict=0.4,
    cohesion_eff=1e3,
    landslides_return_time=1000,
    landslides_on_boundary_nodes=False,
)

- And now we run the model:

In [ ]:
U = 1e-2 # Uplift rate in m/y
timestep = 200  # years
landslides_size_all_steps = []

for i in tqdm(range(50)):
    # Uplift at core nodes
    g.at_node["bedrock__elevation"][g.core_nodes] += U * timestep
    g.at_node["topographic__elevation"][:] = (
        g.at_node["bedrock__elevation"] + g.at_node["soil__depth"]
    )

    # Run the flow router
    fr.run_one_step() # PriorityFloodFlowRouter run
    sp.run_one_step(dt=timestep) # Space run
    hy.run_one_step(dt=timestep) # BerockLandslider run

# Display the final topography.
imshow_grid(
        g,
        "topographic__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Topographic elevation",
        show_elements=True,
    )

<br>
<br/>
Please reuse this material citing Lenard et al., 2023, CSDMS Annual Meeting.<br/>
Author: Sebastien Lenard sebastien.lenard@gmail.com, with available notebooks by Benjamin Campforts<br>
Date: 2023, May.

### Click here to learn about <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>